# wiki scraper and lat/lon

imports

In [1]:
import pandas as pd

set wiki page URL

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

read webpage and store as 'tables'

In [3]:
tables = pd.read_html(url, header=0)

since there are multiple tables, set a list of the headings of the table we're interested in

In [4]:
headings = ['Postcode', 'Borough', 'Neighbourhood']

loop over the tables in the webpage, and if the current table in the loop's headings match the headings we're interested in, we've found the table; thus exit the loop

In [5]:
for table in tables:
    current_headings = table.columns.values[:4]
    if all(current_headings == headings):
        break

filter out postcodes without boroughs

In [6]:
codes = table[table.Borough != 'Not assigned']

create a new dataframe grouping by postcode+borough combinations and aggregating neighborhoods into a comma-separated list

In [7]:
agg = codes.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).to_frame()

reset the index of the aggregated dataframe and save it to a new dataframe

In [8]:
agg2 = agg.reset_index()
agg2

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


##### note
I abaonded geocoder because Anaconda couldn't install it correctly

set URL of lat/lon data

In [9]:
backup_url = 'http://cocl.us/Geospatial_data'

read file and store lat/lon data as 'geo_data'

In [10]:
geo_data = pd.read_csv(backup_url)
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


add the latitude and longitude columns to the aggregated and indexed dataframe

In [11]:
agg2['Latitude'] = geo_data['Latitude']
agg2['Longitude'] = geo_data['Longitude']
agg2

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
